<a href="https://colab.research.google.com/github/mchhour31/NN_work/blob/main/nlp_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [26]:
from sklearn.datasets import fetch_20newsgroups

df_train = fetch_20newsgroups(subset='train', shuffle=True)
X_train = df_train.data
y_train = df_train.target

df_test = fetch_20newsgroups(subset='test', shuffle=True)
X_test = df_test.data
y_test = df_test.target

In [27]:
df_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [28]:
df_test.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

### Naive Approach

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

# bag of words model
cv = CountVectorizer()
X_train_count = cv.fit_transform(X_train)
X_train_count.shape

(11314, 130107)

In [30]:
from sklearn.feature_extraction.text import TfidfTransformer

# tf-idf (penalises repeating words)
tfidf = TfidfTransformer()
X_train_idf = tfidf.fit_transform(X_train_count)
X_train_idf.shape

(11314, 130107)

In [31]:
tfidf.fit_transform(cv.fit_transform(X_test)).shape

(7532, 93564)

In [32]:
from sklearn.naive_bayes import MultinomialNB

# naive bayes classifier
clf = MultinomialNB().fit(X_train_idf, y_train)

## Pipeline of Naive Approach

In [33]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB())
])

text_clf = text_clf.fit(X_train, y_train)
prediction = text_clf.predict(X_test)
print(np.mean(prediction == y_test))

0.7738980350504514


In [35]:
from sklearn.model_selection import GridSearchCV # exhaustive search through values, selects best one (hyperparameter tuning)

parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}

In [37]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [38]:
gs_clf.best_params_

{'clf__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

In [40]:
print(f"Accuracy of Naive Bayes Classifier w/ optimal parameters: {gs_clf.best_score_}")

Accuracy of Naive Bayes Classifier w/ optimal parameters: 0.9157684864695698


## Using SVM for classification

In [53]:
from sklearn.linear_model import SGDClassifier

text_clf_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf-svm', SGDClassifier(loss='modified_huber', penalty='l2', alpha=1e-3, max_iter=1000, random_state=0)) # huber loss (less sensitive to outliers)
])

text_clf_sgd = text_clf_sgd.fit(X_train, y_train)
y_pred = text_clf_sgd.predict(X_test)
print(np.mean(y_test == y_pred))

0.8396176314391928


Hyperparameter Tuning

In [51]:
from sklearn.model_selection import GridSearchCV

parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False),'clf-svm__alpha': (1e-2, 1e-3)}

In [54]:
gcv = GridSearchCV(text_clf_sgd, parameters_svm, n_jobs=-1).fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [55]:
gcv.best_params_

{'clf-svm__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

In [59]:
print(f"Accuracy of SVM Classification Model: {gcv.best_score_}")

Accuracy of SVM Classification Model: 0.9061341622604517
